In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [9]:
!pip install --upgrade scipy

     |████████████████████████████████| 38.1 MB 1.8 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
df = pd.read_csv('music_genre.csv')

In [3]:
df.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [4]:
df.describe()

,instance_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,valence
count,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,55888.396360,44.220420,0.306383,0.558241,2.212526e+05,0.599755,0.181601,0.193896,-9.133761,0.093586,0.456264
std,20725.256253,15.542008,0.341340,0.178632,1.286720e+05,0.264559,0.325409,0.161637,6.162990,0.101373,0.247119
min,20002.000000,0.000000,0.000000,0.059600,-1.000000e+00,0.000792,0.000000,0.009670,-47.046000,0.022300,0.000000
25%,37973.500000,34.000000,0.020000,0.442000,1.748000e+05,0.433000,0.000000,0.096900,-10.860000,0.036100,0.257000
50%,55913.500000,45.000000,0.144000,0.568000,2.192810e+05,0.643000,0.000158,0.126000,-7.276500,0.048900,0.448000
75%,73863.250000,56.000000,0.552000,0.687000,2.686122e+05,0.815000,0.155000,0.244000,-5.173000,0.098525,0.648000
max,91759.000000,99.000000,0.996000,0.986000,4.830606e+06,0.999000,0.996000,1.000000,3.744000,0.942000,0.992000


In [5]:
# Question 1: Hypothsis testing on music genre(hip-hop vs rap) and energy difference
# Part 1: two-tailed test

from scipy.stats import ttest_ind
hip_energy = df.loc[df['music_genre'] == 'Hip-Hop', 'energy']
rap_energy = df.loc[df['music_genre'] == 'Rap', 'energy']
t_score, p_val = ttest_ind(hip_energy, rap_energy, nan_policy='omit')
print('t_score:', t_score, 'p_val:', p_val)

t_score: -2.272140238517617 p_val: 0.023099184902702043


In [6]:
# Question 1: Hypothsis testing on music genre(hip-hop vs rap) and energy difference
# Part 2: one-tailed test
t_score1, p_val1 = ttest_ind(rap_energy, hip_energy, nan_policy='omit', alternative='greater')
print('t_score:', t_score1, 'p_val:', p_val1)

t_score: 2.272140238517617 p_val: 0.011549592451351022


In [7]:
#Question 2: Predict the music genres from the attributes identified before.
#Check missing values
for column in df.columns:
    print("Number of Missing value",column,":", df[column].isnull().values.sum())
print("Data original Shape", df.shape)
print()
df = df.dropna()
print("Total Number of Missing value: ", df.isnull().sum().sum())
print("Data shape after drop missing rows",df.shape)

Number of Missing value instance_id : 5
Number of Missing value artist_name : 5
Number of Missing value track_name : 5
Number of Missing value popularity : 5
Number of Missing value acousticness : 5
Number of Missing value danceability : 5
Number of Missing value duration_ms : 5
Number of Missing value energy : 5
Number of Missing value instrumentalness : 5
Number of Missing value key : 5
Number of Missing value liveness : 5
Number of Missing value loudness : 5
Number of Missing value mode : 5
Number of Missing value speechiness : 5
Number of Missing value tempo : 5
Number of Missing value obtained_date : 5
Number of Missing value valence : 5
Number of Missing value music_genre : 5
Data original Shape (50005, 18)

Total Number of Missing value:  0
Data shape after drop missing rows (50000, 18)


In [8]:
#Question 2: Predict the music genres from the attributes identified before.
#Check duplicated data
print("Duplicate Data: ",df.duplicated().sum())

Duplicate Data:  0


In [9]:
#Question 2: Predict the music genres from the attributes identified before.
#Drop uninformative columns and corrupted column
data = df.drop(columns=['instance_id', "obtained_date", 'tempo'])
data.columns

Index(['artist_name', 'track_name', 'popularity', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'valence',
       'music_genre'],
      dtype='object')

In [10]:
#Question 2: Predict the music genres from the attributes identified before.
#Change the categorical data to numeric values
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['mode'] = label_encoder.fit_transform(data['mode']).astype('float64')
# data['music_genre'] = label_encoder.fit_transform(data['music_genre']).astype('float64')
data['key'] = label_encoder.fit_transform(data['key']).astype('float64')

In [11]:
text = []
numeric = []
for name in data.columns:
    if data[name].dtype == object:
        text.append(name)
    else:
        numeric.append(name)
print("Numeric Data: ", *numeric)
print("Text Columns: ", *text)

Numeric Data:  popularity acousticness danceability duration_ms energy instrumentalness key liveness loudness mode speechiness valence
Text Columns:  artist_name track_name music_genre


In [12]:
#Question 2: Predict the music genres from the attributes identified before.
#process text data
import re
import string

def preprocessing(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+','',text)
    text = re.sub("[%s]" % re.escape(string.punctuation),'',text)
    text = re.sub("\n",'',text)
    text = re.sub('\w*\d\w*','',text)
    
    return(text)
data['track_name'] = data['track_name'].apply(preprocessing)
data['artist_name'] = data['artist_name'].apply(preprocessing)

In [13]:
import nltk
nltk.download('punkt')
sentence = list()
for i in data['artist_name']:
    sentence.append(nltk.word_tokenize(i))
for i in data['track_name']:
    sentence.append(nltk.word_tokenize(i))
    
from gensim.models import Word2Vec
model_w2c=Word2Vec(sentence,window=3,workers=4,min_count=2)

def word_to_bow(text):
    words = nltk.word_tokenize(text)
    bow = 0
    for i in words:
        try:
            bow += sum(model_w2c.wv[i])
        except:
            pass
    return bow
data['track_name'] = data['track_name'].apply(word_to_bow)
data['artist_name'] = data['artist_name'].apply(word_to_bow)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
#Question 2: Predict the music genres from the attributes identified before.
#Change the music_genre to a numerical values
data['music_genre'].replace(to_replace=['Alternative', 'Anime', 'Blues','Classical','Country','Electronic','Hip-Hop','Jazz','Rap','Rock'], value=[0,1, 2, 3,4,5,6,7,8,9], inplace=True)

In [15]:
#Question 2: Predict the music genres from the attributes identified before.
#Model development and evaluation
from sklearn import datasets
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import roc_auc_score

X = data.iloc[:,:14]
y = data.iloc[:,-1]

#Create Test and Train Dataset
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)

#Logistic regression
lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')
lm.fit(X_train, y_train)

#Prediction/classification
Y_predict = lm.predict(X_test)

#Crossvalidation
auc = roc_auc_score(y_test, lm.predict_proba(X_test), multi_class='ovr')
print('AUC:', auc)

AUC: 0.748685759296313


In [16]:
# Question 3: Regression analysis on popularity of different genres
# Load data and pre-process again
df = pd.read_csv('music_genre.csv')
for column in df.columns:
    print("Number of Missing value",column,":", df[column].isnull().values.sum())
print("Data original Shape", df.shape)
print()
df = df.dropna()
df = df[df['tempo'] != '?']
print("Total Number of Missing value: ", df.isnull().sum().sum())
print("Data shape after drop missing rows",df.shape)

df = df.drop(columns=['duration_ms','artist_name','track_name','instance_id', "obtained_date", 'key'])
df['music_genre'] = df['music_genre'].astype('category')
c = df.music_genre.astype('category')
map = dict(enumerate(c.cat.categories))
df['mode'] = df['mode'].astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

Number of Missing value instance_id : 5
Number of Missing value artist_name : 5
Number of Missing value track_name : 5
Number of Missing value popularity : 5
Number of Missing value acousticness : 5
Number of Missing value danceability : 5
Number of Missing value duration_ms : 5
Number of Missing value energy : 5
Number of Missing value instrumentalness : 5
Number of Missing value key : 5
Number of Missing value liveness : 5
Number of Missing value loudness : 5
Number of Missing value mode : 5
Number of Missing value speechiness : 5
Number of Missing value tempo : 5
Number of Missing value obtained_date : 5
Number of Missing value valence : 5
Number of Missing value music_genre : 5
Data original Shape (50005, 18)

Total Number of Missing value:  0
Data shape after drop missing rows (45020, 18)


In [17]:
# Question 3: Regression analysis on popularity of different genres
# Model development, analysis and evaluation
from sklearn.linear_model import LinearRegression
genre = [0,1,2,3,4,5,6,7,8,9]
weight = []
for gen in genre:
  y = df[df['music_genre'] == gen].iloc[:,0:1]
  x = df[df['music_genre'] == gen].iloc[:,1:-1]
  reg = LinearRegression(normalize = True).fit(x, y)
  np.printoptions(suppress=True)
  weight.append(np.round(reg.coef_,2))

dominant_factor = dict()
for i in range(10):
  dominant_factor[map[i]] = x.columns[weight[i].argmax()]
  
print("Contributors' weights for each music genre is as below:")
for i in range(10):
  print(weight[i])

print("\nAnd the biggest contributors for each music genre is as below")
dominant_factor

Contributors' weights for each music genre is as below:
[[ 0.26  1.24 -2.36 -1.35 -1.61  0.19  0.84 -2.27 -0.01  1.96]]
[[ 1.76 -2.11  2.19 -4.47 -1.49  0.2   0.06 -0.89 -0.01 -0.47]]
[[-0.67 -3.64  3.04 -4.53 -5.67 -0.12  0.4  -8.08 -0.    0.13]]
[[-9.920e+00  2.010e+00  1.043e+01  5.590e+00 -2.001e+01 -7.000e-02
   4.200e-01 -3.805e+01 -2.000e-02 -5.720e+00]]
[[ 0.86  0.67  3.93  0.77 -5.51  0.02  0.81 -6.79 -0.01  1.11]]
[[ 1.890e+00  4.270e+00 -1.350e+00 -2.760e+00 -1.420e+00  8.000e-02
   2.000e-01 -1.087e+01  1.000e-02  8.700e-01]]
[[ 1.97  1.85 -9.13 -3.94 -1.38  0.62 -0.26 -4.1   0.    1.63]]
[[ 0.24  6.82 -1.91 -3.98 -1.97  0.02 -0.24 -7.85  0.   -4.3 ]]
[[ 0.79  2.67 -7.49  0.99 -1.81  0.56  0.32 -3.65  0.01 -0.15]]
[[ 1.12  0.64 -1.04 -2.6  -1.76  0.13  0.44  3.39 -0.    1.06]]

And the biggest contributors for each music genre is as below


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pi

{'Alternative': 'valence',
 'Anime': 'energy',
 'Blues': 'energy',
 'Classical': 'energy',
 'Country': 'energy',
 'Electronic': 'danceability',
 'Hip-Hop': 'acousticness',
 'Jazz': 'danceability',
 'Rap': 'danceability',
 'Rock': 'speechiness'}